### network analysis
- 논문 간의 키워드 연관성을 추출해본다.
- 키워드로 논문간의 번들을 형성할 수 있다. 
    - 논문의 트렌드를 형성할 수 있다.

In [1]:
import numpy as np
import pandas as pd
from IPython.display import display
import re

In [2]:
df = pd.read_csv('references.csv')

In [3]:
df.tail()

,infor,refer
1457,KOSPI200 옵션전략지수의 성과\n권순신 | 강병진 | 정재만 | 한국파생상품학...,0
1458,KRX금시장과 국제금융시장의 상호연관성에 관한 연구 : CME선물시장을 중심으로\n...,0
1459,Analytical Approximations of American Call Opt...,0
1460,현금흐름 위험 기반 KOSPI 수익률 횡단면 연구\n문성제 | 송준혁 | 한국파생상...,0
1461,모호성 회피성향이 예방적 노력에 미치는 영향\n홍지민 | 한국파생상품학회 | 선물연...,0


### 참고문헌이 나와있지 않은 논문의 수

In [4]:
len([i for i in df['refer'].values if i == '0'])

242

In [5]:
str(((len([i for i in df['refer'].values if i == '0']) / len(df)) * 100)) + '%'

'16.55266757865937%'

# Tuning_process

#### 우선 information 컬럼을 늘려준다.
- refer 컬럼을 split 하는 과정을 시행한다.

In [6]:
print(len(re.findall('[0-9]{1,}. \[',df['refer'].values[100])))
print(len(re.split('[0-9]{1,}. \[',df['refer'].values[100])))

31
32


In [7]:
head_ls = re.findall('[0-9]{1,}. \[',df['refer'].values[100])

In [8]:
testing_ls = re.split('[0-9]{1,}. \[',df['refer'].values[100])
process_ls = []
for i in range(len(testing_ls)):
    if re.search('\/ [0-9]{,4} \/',testing_ls[i]) : 
        process_ls.append(testing_ls[i])
process_ls[-2:]

['학술지(정기간행물)] - / 1986 / Large Shareholders and Corporate Control (94) : 461 ~ 488,',
 '학술지(정기간행물)] - / 1995 / Do Corporations Award CEO Stock Options Effectively? 39 : 237 ~ 269']

In [9]:
title_ls = []
for i in range(len(process_ls)):
    title_ls.append(head_ls[i] + process_ls[i])
title_ls[-2:]

['30. [학술지(정기간행물)] - / 1986 / Large Shareholders and Corporate Control (94) : 461 ~ 488,',
 '31. [학술지(정기간행물)] - / 1995 / Do Corporations Award CEO Stock Options Effectively? 39 : 237 ~ 269']

### information 컬럼을 늘려주는 function(generalization) 프로세스

In [10]:
#length column 을 만들어주기 위한 프로세스
final_process_ls = []

for i in range(len(df['refer'])):
    try :
        title_ls = []
        if df['refer'].values[i] != '0' : 
            head_ls = re.findall('[0-9]{1,}. \[',df['refer'].values[i])
            testing_ls = re.split('[0-9]{1,}. \[',df['refer'].values[i])
            process_ls = []
            for j in range(len(testing_ls)):
                if re.search('\/ [0-9]{,4} \/',testing_ls[j]) : 
                    process_ls.append(testing_ls[j])
            for k in range(len(process_ls)):
                title_ls.append(head_ls[k] + process_ls[k])
        else : title_ls.append('0')
        final_process_ls.append(title_ls)
    except Exception as e:
        print(i)
        print(len(process_ls),len(head_ls))
        print(e)
        break

In [11]:
df['length'] = [len(i) for i in final_process_ls]

In [13]:
new_df = pd.DataFrame()
new_df['refer'] = [j for i in final_process_ls for j in i]

In [14]:
extending_title_ls = []
for i in range(len(df)):
    for _ in range(df['length'].values[i]):
        extending_title_ls.append(df['infor'].values[i])
print(len(extending_title_ls))
new_df['infor'] = extending_title_ls

37247


In [15]:
new_df = pd.DataFrame()
new_df['infor'] = extending_title_ls
new_df['refer'] = [j for i in final_process_ls for j in i]

In [18]:
set(new_df['infor'].values) == set(df['infor'].values)
#OK
new_df

,infor,refer
0,기업의 소유구조와 기업가치 및 수익률에 대한 연구: 임원지분율을 중심으로\n김병호 ...,0
1,원/달러 현물과 선물간의 일중 사전적 차익거래 수익성\n이재하 | 임상규 | 한국증...,0
2,주식 수익률의 카오스적 특성에 관한 실증연구\n장경천 | 김현석 | 한국증권학회 |...,0
3,한국주식시장에서 셰속투자전략과 반대투자전략의 수익성분석\n이정도 | 안영규 | 한국...,0
4,소유 구조와 기업의 회계적 성과 및 Tobin-Q의 관계에 관한 연구\n박기성 | ...,0
5,지식자산가치 평가모형과 지식자산가치의 기여도에 관한 실증연구\n이원흠 | 최수미 |...,0
6,나스닥증권시장이 한국증시의 가격변동성에 미치는 영향\n김태혁 | 강석규 | 한국증권...,0
7,내부자거래 규제의 경제적 타당성 평가\n최도성 | 고봉찬 | 한국증권학회 | Asi...,0
8,기업의 자산매각 동기와 위기대응-IMF 경제위기를 중심으로-\n위정범 | 한경동 |...,0
9,자산가격변동에 대한 중앙은행의 방응함수 연구\n이민원 | 한국증권학회 | Asia-...,0


# Text Tuning process

# 우선 references 가 아닌 논문의 이름부터 튜닝한다.

infor 컬럼은 | 단위로 잘라줄 수 있고 \n 단위로 잘라줄 수 있다.

In [42]:
for idx in range(len(new_df['infor'].values)):
    if '\n' not in new_df['infor'].values[idx].split('|')[0]:
        print(idx)

In [51]:
new_title_ls = []
for idx in range(len(new_df['infor'].values)):
    testing_ls = new_df['infor'].values[idx].split('|')
    new_title_ls.append([j for i in [i.split('\n') for i in testing_ls] for j in i])
len(new_title_ls) == len(new_df)

True

### 튜닝한 데이터에 대해서 해당 프로세스의 새로운 데이터들을 넣어줄 새로운 데이터 프레임 tuned_df 를 만들어준다.

In [59]:
tuned_df = pd.DataFrame()
tuned_df['title_bundle'] = new_title_ls
tuned_df.tail()

,title_bundle
37242,"[KOSPI200 옵션전략지수의 성과, 권순신 , 강병진 , 정재만 , 한국파..."
37243,"[KRX금시장과 국제금융시장의 상호연관성에 관한 연구 : CME선물시장을 중심으로,..."
37244,[Analytical Approximations of American Call Op...
37245,"[현금흐름 위험 기반 KOSPI 수익률 횡단면 연구, 문성제 , 송준혁 , 한국..."
37246,"[모호성 회피성향이 예방적 노력에 미치는 영향, 홍지민 , 한국파생상품학회 , ..."


### 리스트의 첫 번째 엘리먼트가 항상 논문의 이름인지 확인하는 프로세스가 필요해 보인다.

In [67]:
tuned_df['title_bundle'].values[0]

['기업의 소유구조와 기업가치 및 수익률에 대한 연구: 임원지분율을 중심으로',
 '김병호 ',
 ' 한국증권학회 ',
 ' Asia-Pacific Journal of Financial Studies ',
 ' 30() ',
 ' pp.391~434 ',
 ' 2002.06 ',
 ' 경영학',
 '피인용횟수 : 84']

#### 이름이 넉넉잡아 5글자 이내라고 가정, 검사해본다.

In [69]:
for paper in tuned_df['title_bundle'].values:
    if len(paper[0]) <= 5 : 
        print(paper)
        
# 없으므로 naive 하지만, 각 리스트의 첫 번째 엘리먼트는 페이퍼 이름인 것을 확인할 수 있다.

### paper name 파싱! 

In [70]:
tuned_df['paper_name'] = [i[0] for i in tuned_df['title_bundle'].values]
tuned_df

,title_bundle,paper_name
0,"[기업의 소유구조와 기업가치 및 수익률에 대한 연구: 임원지분율을 중심으로, 김병호...",기업의 소유구조와 기업가치 및 수익률에 대한 연구: 임원지분율을 중심으로
1,"[원/달러 현물과 선물간의 일중 사전적 차익거래 수익성, 이재하 , 임상규 , ...",원/달러 현물과 선물간의 일중 사전적 차익거래 수익성
2,"[주식 수익률의 카오스적 특성에 관한 실증연구, 장경천 , 김현석 , 한국증권학...",주식 수익률의 카오스적 특성에 관한 실증연구
3,"[한국주식시장에서 셰속투자전략과 반대투자전략의 수익성분석, 이정도 , 안영규 , ...",한국주식시장에서 셰속투자전략과 반대투자전략의 수익성분석
4,"[소유 구조와 기업의 회계적 성과 및 Tobin-Q의 관계에 관한 연구, 박기성 ,...",소유 구조와 기업의 회계적 성과 및 Tobin-Q의 관계에 관한 연구
5,"[지식자산가치 평가모형과 지식자산가치의 기여도에 관한 실증연구, 이원흠 , 최수미...",지식자산가치 평가모형과 지식자산가치의 기여도에 관한 실증연구
6,"[나스닥증권시장이 한국증시의 가격변동성에 미치는 영향, 김태혁 , 강석규 , 한...",나스닥증권시장이 한국증시의 가격변동성에 미치는 영향
7,"[내부자거래 규제의 경제적 타당성 평가, 최도성 , 고봉찬 , 한국증권학회 , ...",내부자거래 규제의 경제적 타당성 평가
8,"[기업의 자산매각 동기와 위기대응-IMF 경제위기를 중심으로-, 위정범 , 한경동...",기업의 자산매각 동기와 위기대응-IMF 경제위기를 중심으로-
9,"[자산가격변동에 대한 중앙은행의 방응함수 연구, 이민원 , 한국증권학회 , As...",자산가격변동에 대한 중앙은행의 방응함수 연구


#### 리스트에서 두 번째부터 `한국증권학회` , `한국재무관리학회` , `한국파생상품학회` , `한국재무학회` 가 나오는 엘리먼트의 리스트 인덱스 전까지는 이름 즉, 저자이다.

In [71]:
tuned_df['title_bundle'].values[0]

['기업의 소유구조와 기업가치 및 수익률에 대한 연구: 임원지분율을 중심으로',
 '김병호 ',
 ' 한국증권학회 ',
 ' Asia-Pacific Journal of Financial Studies ',
 ' 30() ',
 ' pp.391~434 ',
 ' 2002.06 ',
 ' 경영학',
 '피인용횟수 : 84']

##### 우선 한국증권학회 , 한국재무관리학회 , 한국파생상품학회 , 한국재무학회 가 나오는 인덱스를 찾는다.

In [92]:
testing_ls = []
for paper in tuned_df['title_bundle'].values : 
    for idx in range(len(paper)):
        if re.findall('한국증권학회|한국재무관리학회|한국파생상품학회|한국재무학회',paper[idx]) : 
            testing_ls.append(idx)
            break

In [94]:
institution_ls = []
for idx in range(len(tuned_df['title_bundle'].values)):
    institution_ls.append(tuned_df['title_bundle'].values[idx][testing_ls[idx]])

In [95]:
institution_ls[-2:]

[' 한국파생상품학회 ', ' 한국파생상품학회 ']

In [96]:
for i in institution_ls : 
    if not re.findall('한국증권학회|한국재무관리학회|한국파생상품학회|한국재무학회',i):
        print(i)

# 아무것도 출력되지 않으므로 institution_ls 에는 발행기관명만 있다는 것을 알 수 있다.

In [97]:
tuned_df['institution'] = institution_ls
tuned_df

,title_bundle,paper_name,institution
0,"[기업의 소유구조와 기업가치 및 수익률에 대한 연구: 임원지분율을 중심으로, 김병호...",기업의 소유구조와 기업가치 및 수익률에 대한 연구: 임원지분율을 중심으로,한국증권학회
1,"[원/달러 현물과 선물간의 일중 사전적 차익거래 수익성, 이재하 , 임상규 , ...",원/달러 현물과 선물간의 일중 사전적 차익거래 수익성,한국증권학회
2,"[주식 수익률의 카오스적 특성에 관한 실증연구, 장경천 , 김현석 , 한국증권학...",주식 수익률의 카오스적 특성에 관한 실증연구,한국증권학회
3,"[한국주식시장에서 셰속투자전략과 반대투자전략의 수익성분석, 이정도 , 안영규 , ...",한국주식시장에서 셰속투자전략과 반대투자전략의 수익성분석,한국증권학회
4,"[소유 구조와 기업의 회계적 성과 및 Tobin-Q의 관계에 관한 연구, 박기성 ,...",소유 구조와 기업의 회계적 성과 및 Tobin-Q의 관계에 관한 연구,한국증권학회
5,"[지식자산가치 평가모형과 지식자산가치의 기여도에 관한 실증연구, 이원흠 , 최수미...",지식자산가치 평가모형과 지식자산가치의 기여도에 관한 실증연구,한국증권학회
6,"[나스닥증권시장이 한국증시의 가격변동성에 미치는 영향, 김태혁 , 강석규 , 한...",나스닥증권시장이 한국증시의 가격변동성에 미치는 영향,한국증권학회
7,"[내부자거래 규제의 경제적 타당성 평가, 최도성 , 고봉찬 , 한국증권학회 , ...",내부자거래 규제의 경제적 타당성 평가,한국증권학회
8,"[기업의 자산매각 동기와 위기대응-IMF 경제위기를 중심으로-, 위정범 , 한경동...",기업의 자산매각 동기와 위기대응-IMF 경제위기를 중심으로-,한국증권학회
9,"[자산가격변동에 대한 중앙은행의 방응함수 연구, 이민원 , 한국증권학회 , As...",자산가격변동에 대한 중앙은행의 방응함수 연구,한국증권학회


### 이번에는 이름을 해본다. 이름은 두번째 요소부터 발행기관명 사이의 엘리먼트이다. 발행기관명의 인덱스는 testing_ls에 넣어두었다.

In [104]:
name_ls = []
for idx,val in enumerate(tuned_df['title_bundle']):
    name_ls.append(val[1:testing_ls[idx]])
name_ls = [','.join(i) for i in name_ls]
# 리스트를 문자열 형식으로 바꿔준다.

In [105]:
tuned_df['author'] = name_ls

In [106]:
tuned_df

,title_bundle,paper_name,institution,author
0,"[기업의 소유구조와 기업가치 및 수익률에 대한 연구: 임원지분율을 중심으로, 김병호...",기업의 소유구조와 기업가치 및 수익률에 대한 연구: 임원지분율을 중심으로,한국증권학회,김병호
1,"[원/달러 현물과 선물간의 일중 사전적 차익거래 수익성, 이재하 , 임상규 , ...",원/달러 현물과 선물간의 일중 사전적 차익거래 수익성,한국증권학회,"이재하 , 임상규"
2,"[주식 수익률의 카오스적 특성에 관한 실증연구, 장경천 , 김현석 , 한국증권학...",주식 수익률의 카오스적 특성에 관한 실증연구,한국증권학회,"장경천 , 김현석"
3,"[한국주식시장에서 셰속투자전략과 반대투자전략의 수익성분석, 이정도 , 안영규 , ...",한국주식시장에서 셰속투자전략과 반대투자전략의 수익성분석,한국증권학회,"이정도 , 안영규"
4,"[소유 구조와 기업의 회계적 성과 및 Tobin-Q의 관계에 관한 연구, 박기성 ,...",소유 구조와 기업의 회계적 성과 및 Tobin-Q의 관계에 관한 연구,한국증권학회,박기성
5,"[지식자산가치 평가모형과 지식자산가치의 기여도에 관한 실증연구, 이원흠 , 최수미...",지식자산가치 평가모형과 지식자산가치의 기여도에 관한 실증연구,한국증권학회,"이원흠 , 최수미"
6,"[나스닥증권시장이 한국증시의 가격변동성에 미치는 영향, 김태혁 , 강석규 , 한...",나스닥증권시장이 한국증시의 가격변동성에 미치는 영향,한국증권학회,"김태혁 , 강석규"
7,"[내부자거래 규제의 경제적 타당성 평가, 최도성 , 고봉찬 , 한국증권학회 , ...",내부자거래 규제의 경제적 타당성 평가,한국증권학회,"최도성 , 고봉찬"
8,"[기업의 자산매각 동기와 위기대응-IMF 경제위기를 중심으로-, 위정범 , 한경동...",기업의 자산매각 동기와 위기대응-IMF 경제위기를 중심으로-,한국증권학회,"위정범 , 한경동"
9,"[자산가격변동에 대한 중앙은행의 방응함수 연구, 이민원 , 한국증권학회 , As...",자산가격변동에 대한 중앙은행의 방응함수 연구,한국증권학회,이민원


### 이번에는 저널명이다 testing_ls에서 1을 더해준다.

In [109]:
journal_ls = []
for idx,val in enumerate(tuned_df['title_bundle']):
    journal_ls.append(val[testing_ls[idx]+1])

In [112]:
tuned_df['journal'] = journal_ls

In [113]:
tuned_df

,title_bundle,paper_name,institution,author,journal
0,"[기업의 소유구조와 기업가치 및 수익률에 대한 연구: 임원지분율을 중심으로, 김병호...",기업의 소유구조와 기업가치 및 수익률에 대한 연구: 임원지분율을 중심으로,한국증권학회,김병호,Asia-Pacific Journal of Financial Studies
1,"[원/달러 현물과 선물간의 일중 사전적 차익거래 수익성, 이재하 , 임상규 , ...",원/달러 현물과 선물간의 일중 사전적 차익거래 수익성,한국증권학회,"이재하 , 임상규",Asia-Pacific Journal of Financial Studies
2,"[주식 수익률의 카오스적 특성에 관한 실증연구, 장경천 , 김현석 , 한국증권학...",주식 수익률의 카오스적 특성에 관한 실증연구,한국증권학회,"장경천 , 김현석",Asia-Pacific Journal of Financial Studies
3,"[한국주식시장에서 셰속투자전략과 반대투자전략의 수익성분석, 이정도 , 안영규 , ...",한국주식시장에서 셰속투자전략과 반대투자전략의 수익성분석,한국증권학회,"이정도 , 안영규",Asia-Pacific Journal of Financial Studies
4,"[소유 구조와 기업의 회계적 성과 및 Tobin-Q의 관계에 관한 연구, 박기성 ,...",소유 구조와 기업의 회계적 성과 및 Tobin-Q의 관계에 관한 연구,한국증권학회,박기성,Asia-Pacific Journal of Financial Studies
5,"[지식자산가치 평가모형과 지식자산가치의 기여도에 관한 실증연구, 이원흠 , 최수미...",지식자산가치 평가모형과 지식자산가치의 기여도에 관한 실증연구,한국증권학회,"이원흠 , 최수미",Asia-Pacific Journal of Financial Studies
6,"[나스닥증권시장이 한국증시의 가격변동성에 미치는 영향, 김태혁 , 강석규 , 한...",나스닥증권시장이 한국증시의 가격변동성에 미치는 영향,한국증권학회,"김태혁 , 강석규",Asia-Pacific Journal of Financial Studies
7,"[내부자거래 규제의 경제적 타당성 평가, 최도성 , 고봉찬 , 한국증권학회 , ...",내부자거래 규제의 경제적 타당성 평가,한국증권학회,"최도성 , 고봉찬",Asia-Pacific Journal of Financial Studies
8,"[기업의 자산매각 동기와 위기대응-IMF 경제위기를 중심으로-, 위정범 , 한경동...",기업의 자산매각 동기와 위기대응-IMF 경제위기를 중심으로-,한국증권학회,"위정범 , 한경동",Asia-Pacific Journal of Financial Studies
9,"[자산가격변동에 대한 중앙은행의 방응함수 연구, 이민원 , 한국증권학회 , As...",자산가격변동에 대한 중앙은행의 방응함수 연구,한국증권학회,이민원,Asia-Pacific Journal of Financial Studies


#### 나머지 4개의 요소를 보게 되면, 33(1)는 무엇인지 모르겠고 나머지는 각각 페이지 수와, 논문 발행 년도 , 논문의 분야 카테고리 , 마지막으로 피인용횟수가 되겠다. 각각 분배해주자.

In [118]:
tuned_df['title_bundle'].values[100]

['해외전환사채 전환권가치의 정보효과',
 '정무권 ',
 ' 한국증권학회 ',
 ' Asia-Pacific Journal of Financial Studies ',
 ' 33(1) ',
 ' pp.33~68 ',
 ' 2004.03 ',
 ' 경영학',
 '피인용횟수 : 3']

### 뭔지 모르겠지만 넣어준다.

In [119]:
dont_know_ls = []
for idx,val in enumerate(tuned_df['title_bundle']):
    dont_know_ls.append(val[-5])

tuned_df['dont_know'] = dont_know_ls

In [133]:
dont_know_ls[100]

' 33(1) '

In [135]:
[i for i in dont_know_ls if not re.findall(' [0-9]{2}\(.*\)',i)]

[]

### 페이지 수

In [120]:
page_ls = []
for idx,val in enumerate(tuned_df['title_bundle']):
    page_ls.append(val[-4])

tuned_df['page'] = page_ls

In [137]:
[i for i in page_ls if not re.findall(' pp.',i)]

[]

### 발행 시점

In [121]:
year_ls = []
for idx,val in enumerate(tuned_df['title_bundle']):
    year_ls.append(val[-3])

tuned_df['year'] = year_ls

In [141]:
[i for i in year_ls if not re.findall(' 20',i)]

[]

### 논문 분야

In [122]:
field_ls = []
for idx,val in enumerate(tuned_df['title_bundle']):
    field_ls.append(val[-2])

tuned_df['field'] = field_ls

In [144]:
[i for i in field_ls if not re.findall('경영학',i)]

[]

### 피인용 횟수

In [123]:
cited_ls = []
for idx,val in enumerate(tuned_df['title_bundle']):
    cited_ls.append(val[-1])

tuned_df['cited'] = cited_ls

In [148]:
[i for i in cited_ls if not re.findall('피인용횟수',i)]

[]

# 이렇게 해서 기존 df 라는 데이터 프레임에서 infor 컬럼의 튜닝을 끝냈다.

In [154]:
final_tuned_df = tuned_df.iloc[:,1:]
final_tuned_df.tail()

,paper_name,institution,author,journal,dont_know,page,year,field,cited
37242,KOSPI200 옵션전략지수의 성과,한국파생상품학회,"권순신 , 강병진 , 정재만",선물연구,26(2),pp.183~216,2018.05,경영학,피인용횟수 : 0
37243,KRX금시장과 국제금융시장의 상호연관성에 관한 연구 : CME선물시장을 중심으로,한국파생상품학회,"염명훈 , 김지훈",선물연구,26(3),pp.345~369,2018.08,경영학,피인용횟수 : 0
37244,Analytical Approximations of American Call Opt...,한국파생상품학회,배광일,선물연구,26(3),pp.283~310,2018.08,경영학,피인용횟수 : 0
37245,현금흐름 위험 기반 KOSPI 수익률 횡단면 연구,한국파생상품학회,"문성제 , 송준혁",선물연구,26(3),pp.311~343,2018.08,경영학,피인용횟수 : 0
37246,모호성 회피성향이 예방적 노력에 미치는 영향,한국파생상품학회,홍지민,선물연구,26(3),pp.371~389,2018.08,경영학,피인용횟수 : 0


In [153]:
final_tuned_df.to_csv('tuned_dataframe_infor.csv',index=False)